# Análisis de costos de cursos en el mercado

# Instalación e importación de librerías

In [12]:
import pandas as pd
import numpy as np
import os
from hermetrics.levenshtein import Levenshtein
import re
from tqdm import tqdm
from hermetrics.levenshtein import Levenshtein

# Rutas de archivos

In [5]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))
path_db = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))

### Palabras clave

In [7]:
keyword = ['Python', 'programación web', 'paginas web', 'datos', 'Excel', 'Word', 'power point', 'BI', 'inteligencia de negocios', 'powerBI', 
'big data', 'programación', 'marketing', 'web', 'html', 'css',  'java', 'javascript', 'base de datos', 'redes', 'telecomunicaciones', 'Android', 'IOS', 'ARDUINO'
'Business Intelligence', 
'Desarrollo de software',
'Fibra Óptica',
'Gerencia de sistemas',
'Hackeo Ético',
'Informática Forense',
'Linux', 'WINDOWS'
'Office',
'PHP',
'Project',
'software R', 'Tableu', 'rapidminer', 
'Mysql', 'sql', 'sqlserver', 'Oracle', 'postgresql',
'Seguridad informática',
'Aulas virtuales', 'Moodle', 'software',
'Wordpress', 'joomla', 'xoops', 'drupal'
]

keyword = np.char.lower(np.array(keyword))

In [30]:
df_trate = pd.read_csv(os.path.join(path_db,'costos_estimados_cursos_perfiles.csv'))

In [40]:
def get_costo2(df_curso_perfil,df_info_cursos_edu,df_trate):
    lev = Levenshtein()
    size_tt = df_trate.shape[0]
    size_cp = df_curso_perfil.shape[0]
    size_ic = df_info_cursos_edu.shape[0]
    for i in tqdm(range(size_tt,size_cp)):
        res_value = 0
        if df_curso_perfil.iloc[i]['tipo'] == 'curso':
            valor_comparacion = 0
            index_valor = 0
            
            variables_similaridad_select = {}
            for j in range(0,size_ic):
                # Variables de similaridad
                #Comprobar la similalidad del curso
                curso_fuente = df_curso_perfil.iloc[i]['curso_perfil']
                curso_comparar = df_info_cursos_edu.iloc[j]['cursos']
                val_sim = 0
                try:
                    val_sim = lev.similarity(curso_fuente,curso_comparar,(1.25,4,1.5))
                except Exception as e:
                    val_sim = 0
                variables_similaridad_select['precio_por_hora'] = val_sim

                #Comprobar la similalidad por modalidad
                mod_fuente = df_curso_perfil.iloc[i]['modalidad']
                mod_comparar = df_info_cursos_edu.iloc[j]['modalidad_set']
                variables_similaridad_select['precio_mean_by_mod']  = 1 if mod_fuente == mod_comparar else 0

                #Comprobar la similalidad por area
                area_fuente = df_curso_perfil.iloc[i]['area_familia']
                area_comparar = df_info_cursos_edu.iloc[j]['area_familia']
                variables_similaridad_select['precio_mean_by_area'] = 1 if area_fuente == area_comparar else 0.5 

                #Comprobar la similalidad por ciudad
                ciu_fuente = df_curso_perfil.iloc[i]['canton']
                ciu_comparar = df_info_cursos_edu.iloc[j]['ciudad']
                variables_similaridad_select['precio_mean_by_ciudad']  = 1 if ciu_fuente in ciu_comparar else 0

                #Comprobar la similalidad por modalidad y area
                variables_similaridad_select['precio_mean_by_ma']  = 2 if area_fuente == area_comparar and mod_fuente == mod_comparar else 0

                #Comprobar la similalidad por area y ciudad
                variables_similaridad_select['precio_mean_by_ac']  = 2 if area_fuente == area_comparar and ciu_fuente == ciu_comparar else 0

                #Comprobar la similalidad por modalidad y ciudad
                variables_similaridad_select['precio_mean_by_mc']  = 2 if mod_fuente == mod_comparar and ciu_fuente == ciu_comparar else 0

                #Comprobar la similalidad por modalidad, area y ciudad
                variables_similaridad_select['precio_mean_by_mac']  = 3 if mod_fuente == mod_comparar and ciu_fuente == ciu_comparar and area_fuente == area_comparar else 0

                 #Comprobar la similalidad por carga horaria
                hora_fuente = df_curso_perfil.iloc[i]['carga_horaria']
                hora_comparar = df_info_cursos_edu.iloc[j]['carga_horaria']
                variables_similaridad_select['precio_by_horas']  = 1 if hora_fuente == hora_comparar else 0
                aux_valor = 0
                for var in variables_similaridad_select.values():
                    aux_valor += var
                if  aux_valor >=  valor_comparacion:
                    valor_comparacion = aux_valor
                    index_valor = j
                    res_value = np.nan
                    while np.isnan(res_value):

                        max_key = max(variables_similaridad_select,key=variables_similaridad_select.get)
                        if max_key == 'precio_by_horas' or  max_key == 'precio_mean_by_mac':  
                            res_value = df_info_cursos_edu.iloc[index_valor]['precio_por_hora']
                        else:
                            res_value = df_info_cursos_edu.iloc[index_valor][max_key]

                        #df.iloc[i]['costo'] = res_value
                        variables_similaridad_select[max_key] = -1
        else:
            res_value=df_curso_perfil.iloc[i]['costo']
        id_curso_perfil=df_curso_perfil.iloc[i]['id_curso_perfil']
        df_aux = pd.DataFrame(data=[[id_curso_perfil,res_value]],columns=['id_curso_perfil','precio_estimado_por_hora'])
        df_aux.to_csv(os.path.join(path_db,'costos_estimados_cursos_perfiles.csv'),mode='a', header=not os.path.exists(os.path.join(path_db,'costos_estimados_cursos_perfiles.csv')),index=False)

    return df_aux


In [41]:
df_curso_perfil = pd.read_csv(os.path.join(path_dwh,'dim_curso_perfil.csv'))
df_info_cursos_edu = pd.read_csv(os.path.join(path_db,'costos_cursos_educacion_continua.csv'))

In [42]:
arr_costo = get_costo2(df_curso_perfil,df_info_cursos_edu,df_trate)

100%|████████████████████████████████████████████████████████████████████████████| 9337/9337 [2:31:05<00:00,  1.03it/s]


In [ ]:
df_aux2 = pd.DataFrame(data=arr_costo)
df_aux2 = df_aux2.T
df_aux2 = df_aux2.rename(columns={0:'id_curso_perfil',1:'precio_estimado_por_hora'})
df_aux2['id_curso_perfil'] = df_aux2['id_curso_perfil'].astype('int64')

In [23]:
df_aux2

,id_curso_perfil,precio_estimado_por_hora
0,0,1.71
1,1,1.71
2,2,1.71
3,3,1.71
4,4,1.13
5,5,1.12
6,6,1.13
7,7,1.13
8,8,1.13
9,9,1.71


In [21]:
df_aux = df_curso_perfil.iloc[:20,:]
df_aux=pd.merge(df_aux,df_aux2,on='id_curso_perfil',how='inner')

In [22]:
df_aux

,id_curso_perfil,tipo,razon_social,area_familia,especialidad_sector,curso_perfil,modalidad,carga_horaria,costo,canton,precio_estimado_por_hora
0,0,curso,mogrovejo narvaez jhoni william,mecánica automotriz,mecánica general,mecánica automotriz,presencial,120,0.0,cuenca,1.71
1,1,curso,mogrovejo narvaez jhoni william,artes y artesanía,peluquería y belleza barbería y estilismo,belleza integral,presencial,120,0.0,cuenca,1.71
2,2,curso,mogrovejo narvaez jhoni william,artes y artesanía,cosmetología,cosmetología,presencial,120,0.0,cuenca,1.71
3,3,curso,mogrovejo narvaez jhoni william,mecánica automotriz,carrocería,latonería,presencial,120,0.0,cuenca,1.71
4,4,curso,mogrovejo narvaez jhoni william,mecánica automotriz,mecánica general,fundamentos teóricos mecánica automotriz,virtual,40,0.0,cuenca,1.13
5,5,curso,mogrovejo narvaez jhoni william,artes y artesanía,peluquería y belleza barbería y estilismo,fundamentos teóricos belleza integral,virtual,40,0.0,cuenca,1.12
6,6,curso,mogrovejo narvaez jhoni william,artes y artesanía,cosmetología,fundamentos teóricos cosmetología,virtual,40,0.0,cuenca,1.13
7,7,curso,mogrovejo narvaez jhoni william,mecánica automotriz,carrocería,fundamentos teóricos latonería,virtual,40,0.0,cuenca,1.13
8,8,curso,mogrovejo narvaez jhoni william,servicios socioculturales y a la comunidad,gestión cultural,guitarra y arreglos,virtual,40,0.0,cuenca,1.13
9,9,curso,mogrovejo narvaez jhoni william,electricidad y electrónica,electrodomésticos,reparación de refrigeradoras,presencial,40,0.0,cuenca,1.71


In [8]:
lev = Levenshtein()

In [9]:
stopwords = open(os.path.join(path_ppd,'spanish.txt'),"r", encoding='utf-8')
stopwords = stopwords.readlines()
stopwords = [x.replace('\n','') for x in stopwords]

In [10]:
def normalizar(text):
    puntuacion = '!"#$%&;\'()*+,-./?@[\]^_`{|}~()' 
    text = text.replace('Á','A')
    text = text.replace('É','E')
    text = text.replace('Í','I')
    text = text.replace('Ó','O')
    text = text.replace('Ú','U')
    text = text.strip().lower()
    text = text.split(' ')
    re_punc = re.compile('[%s]' % re.escape(puntuacion))
    stripped = [re_punc.sub(' ', w) for w in text]
    text = ' '.join(stripped)
    text = text.split(' ')
    new_text = []
    for i in range(len(text)):
        char = text[i]
        if char not in stopwords:
            new_text.append(text[i])
    new_text = ' '.join(new_text)
            
    return new_text

In [208]:
#Filtrar las areas de Tecnologia y TICs
df_dim_curso_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[(df_dim_curso_perfil['area_familia']=='información y comunicación')|(df_dim_curso_perfil['area_familia'] == 'tecnologías de la información y comunicación')]
data_1= df_dim_curso_perfil.groupby(['curso_perfil','modalidad']).count().reset_index()
data_1 = data_1[['curso_perfil','modalidad']]

df_dim_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oec','dim_perfil.csv'))
df_dim_perfil = df_dim_perfil[df_dim_perfil['familia']=='tecnología: hardware y software (incluye tic)']
data_2 = df_dim_perfil.groupby('perfil').count().reset_index()
data_2['modalidad'] = 'desconocida'
data_2 = data_2[['perfil','modalidad']]

df_dim_curso = pd.read_csv(os.path.join(path_dwh,'datamart_ci','dim_curso.csv'))
df_dim_curso = df_dim_curso[df_dim_curso['area']=='tecnologías de la información y comunicación']
data_3 = df_dim_curso.groupby('curso').count().reset_index()
data_3['modalidad'] = 'desconocida'
data_3 = data_3[['curso','modalidad']]

arr_dims_curso_perfil = [df_dim_curso_perfil,df_dim_perfil,df_dim_curso]

In [216]:
data = np.concatenate((data_1.values,data_2.values,data_3.values),axis=0)
data = pd.DataFrame(data=data,columns=['curso_perfil','modalidad'])
data = data.drop_duplicates()
data['aux_2_curso_perfil'] = data['curso_perfil'].str.cat(data['modalidad'],sep=' | ')
data['aux_curso_perfil'] = data['curso_perfil'].apply(lambda x: normalizar(x))

In [217]:
df_coec = pd.read_csv(os.path.join(path_ppd,'costos_cursos_educacion_continua.csv'))
df_ccce = pd.read_csv(os.path.join(path_ppd,'costos_de_cursos_cec_epn.csv'))
df_coec = pd.concat([df_coec,df_ccce])
df_coec['aux_2_curso'] = df_coec['curso'].str.cat(df_coec['modalidad'],sep=' | ')
df_coec['aux_curso'] = df_coec['curso'].apply(lambda x: normalizar(x))

In [219]:
arr = []

for i in range(len(df_coec['aux_curso'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(data['aux_curso_perfil'])):
        curso_fuente = df_coec['aux_curso'].iloc[i]
        curso_comparar = data['aux_curso_perfil'].iloc[j]
        similary = lev.similarity(curso_fuente,curso_comparar,(1,1.5,1.25))
        aux_similary=0
        arr_curso = curso_comparar.split(' ')
        if curso_comparar in curso_fuente :
            similary = 1.0
        else:
            for word in arr_curso:
                if word in curso_fuente:
                    aux_similary+=1/len(arr_curso)
            if data['modalidad'].iloc[j] == df_coec['modalidad'].iloc[i]:
              aux_similary += aux_similary/2
        if aux_similary > similary and aux_similary > 2/len(arr_curso):
            similary = aux_similary
        sub_arr.append(similary)
    arr.append(sub_arr)


In [220]:
res = pd.DataFrame(data=arr,index=df_coec['aux_2_curso'],columns=data['aux_2_curso_perfil'])

In [227]:

arr_dims_cols = ['curso_perfil','perfil','curso']
arr_datamarts = ['datamart_oc','datamart_oec','datamart_ci']
arr_files = ['dim_curso_perfil.csv','dim_perfil.csv','dim_curso.csv']
arr_df_final_dims = []
for i in range(len(arr_dims_curso_perfil)):
  df_dim = arr_dims_curso_perfil[i]
  df_dim['curso_modalidad'] = df_dim[arr_dims_cols[i]].str.cat(df_dim['modalidad'],sep=' | ')
  arr_curso_fuente = []
  for j in range(arr_dims_curso_perfil[i].shape[0]):
      query = df_dim['curso_modalidad'].iloc[j]
      val = res[res[query]==res[query].max()][query].index.values[0]
      arr_curso_fuente.append(df_coec[df_coec['aux_2_curso']==val][['curso','precio_por_hora']].values[0])
  df_curso_fuente_precio = pd.DataFrame(data=arr_curso_fuente,columns=['curso_fuente','precio_por_hora'])
  df_res_corr = pd.DataFrame(data=[df_dim[arr_dims_cols[i]].values,df_dim['modalidad'].values,df_curso_fuente_precio['curso_fuente'].values,df_curso_fuente_precio['precio_por_hora'].values])
  df_res_corr = df_res_corr.transpose()
  df_res_corr = df_res_corr.rename(columns={0:arr_dims_cols[i],1:'modalidad',2:'curso_fuente',3:'precio_por_hora'})
  df_dim = pd.read_csv(os.path.join(path_dwh,arr_datamarts[i],arr_files[i]))
  df_dim= pd.merge(df_dim,df_res_corr,left_on=[arr_dims_cols[i],'modalidad'],right_on=[arr_dims_cols[i],'modalidad'],how='left')
  df_dim['costo'] = df_dim['carga_horaria']*df_dim['precio_por_hora']
  df_dim = df_dim.drop(['curso_fuente','precio_por_hora'], axis=1)
  df_dim.to_csv(os.path.join(path_dwh,arr_datamarts[i],arr_files[i]),index=False)
  arr_df_final_dims.append(df_dim)
